In [1]:
import pandas_datareader as web
import quandl as quandl
import wrds as wrds
# storage and operations
import pandas as pd
import numpy as np
import datetime
from pathlib import Path
import seaborn as sns

# stats
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
from scipy import stats
from scipy.stats.mstats import winsorize
import os
from tqdm import tqdm
from zipfile import ZipFile
import seaborn
import matplotlib.pyplot as plt
import linearmodels as lm
from sklearn.linear_model import LinearRegression
from scipy.optimize import minimize

#prtfolio optimization:
# libraries we might use for testing
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tsa.seasonal import seasonal_decompose, STL
import linearmodels as lm
from linearmodels.panel import compare
import time
import quantstats as qs

from datetime import datetime


path_factors = Path(r'C:\Users\mauri\FSoF\Frankfurt School of Finance - Master Thesis\Master Thesis - Data\Factors')
path_wrds = Path(r'C:\Users\mauri\FSoF\Frankfurt School of Finance - Master Thesis\Master Thesis - Data\WRDS')
path_ml = Path(r'C:\Users\mauri\FSoF\Frankfurt School of Finance - Master Thesis\Master Thesis - Machine Learning Models')

factor_data = path_factors / 'factor_data.h5'
macro_data = path_factors / 'macro_data.h5'
wrds_data = path_wrds/'wrds_data.h5'

wrds_data_raw_csv = path_wrds / "wrds_data.csv"

wrds_data_true_csv = path_wrds / "wrds_data_true.csv"

wrds_data_abs_csv = path_wrds / "wrds_data_abs.csv"

wrds_data_complete_abs_csv = path_wrds / "wrds_data_complete_abs.csv"

daily_wrds_data_csv = path_wrds / 'daily_wrds_data.csv'

#connecting to WRDS
db = wrds.Connection(wrds_username='mauricematthias')

Loading library list...
Done


In [2]:
macro_factors = pd.read_hdf(macro_data, 'macro_factors_monthly')
macro_factors = macro_factors.set_index('date')
macro_factors

,D12,E12,b/m,tbl,lty,ntis,Rfree,infl,ltr,corpr,...,DDURRG3M086SBEA,DNDGRG3M086SBEA,DSERRG3M086SBEA,CES0600000008,CES2000000008,CES3000000008,DTCOLNVHFNM,DTCTHFNM,INVEST,VIXCLSx
date,,,,,,,,,,,,,,,,,,,,,
1977-07,4.406670,10.516700,0.896783,0.0519,0.0768,0.033182,0.0042,0.004942,-0.0070,-0.0005,...,85.536,35.172,24.873,5.97,7.54,5.58,12900.00,35574.00,230.3024,15.3617
1977-08,4.453330,10.613300,0.926534,0.0549,0.0754,0.033750,0.0044,0.003279,0.0198,0.0136,...,85.817,35.270,25.027,5.99,7.56,5.61,13058.00,36030.00,230.1719,14.4929
1977-09,4.500000,10.710000,0.942263,0.0581,0.0764,0.032368,0.0043,0.003268,-0.0029,-0.0022,...,85.946,35.317,25.173,6.03,7.62,5.65,13177.00,36241.00,230.9649,14.0867
1977-10,4.556670,10.770000,0.975377,0.0616,0.0781,0.032775,0.0049,0.003257,-0.0093,-0.0038,...,86.439,35.456,25.293,6.08,7.66,5.69,14686.00,37965.00,229.8558,15.4536
1977-11,4.613330,10.830000,0.962034,0.0610,0.0777,0.029217,0.0050,0.004870,0.0093,0.0061,...,86.802,35.699,25.434,6.11,7.68,5.72,14975.00,38553.00,231.6455,19.6377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08,59.128628,98.556667,0.235975,0.0010,0.0065,-0.008504,0.0001,0.003153,-0.0349,-0.0488,...,86.662,98.675,120.600,25.50,29.43,22.85,344023.25,726723.85,4385.2431,22.8791
2020-09,58.851237,98.220000,0.241482,0.0011,0.0068,-0.005698,0.0001,0.001393,0.0080,0.0041,...,86.616,98.508,120.964,25.46,29.11,23.00,347627.43,730734.42,4452.0419,27.5874
2020-10,58.660440,96.856667,0.253146,0.0010,0.0079,-0.001895,0.0001,0.000415,-0.0238,-0.0190,...,86.532,98.398,121.094,25.54,29.31,22.99,348262.68,730398.69,4514.5279,29.4362


In [3]:
monthly_factors = pd.read_hdf(factor_data, 'all_factors_monthly')
monthly_factors

,0trade,abr_1,abr_12,abr_6,aci,adm,almq_1,almq_12,almq_6,amihud,...,spq_12,spq_6,sturn,sue_1,sue_6,ta,tbiq_12,tbiq_6,vfp,vhp
date,,,,,,,,,,,,,,,,,,,,,
1977-07,0.018692,-0.004539,-0.000145,-0.000543,-0.021161,0.012778,-0.016623,-0.011760,-0.012382,0.013096,...,-0.002878,-0.003610,0.025480,0.025039,0.020080,-0.006119,0.014968,0.012904,0.008684,-0.007701
1977-08,-0.005823,0.016975,-0.001645,-0.003775,-0.003913,-0.003893,-0.016834,-0.028656,-0.031445,0.001388,...,-0.009160,-0.010745,-0.000193,0.015148,0.015928,0.016202,-0.003095,-0.002748,-0.005812,-0.029833
1977-09,0.009677,0.017575,0.006374,0.008475,-0.002640,-0.007515,-0.005426,-0.014506,-0.009094,0.011608,...,-0.019648,-0.019498,0.004160,0.020957,0.018645,0.010797,0.013106,0.012017,-0.009841,0.018992
1977-10,0.011402,-0.006925,-0.007474,-0.004100,-0.005689,0.012399,0.015994,0.010833,0.013832,0.016949,...,-0.000928,-0.000866,0.010722,0.000512,0.005392,0.007045,0.008343,0.006349,-0.107805,0.011240
1977-11,-0.030581,0.031692,0.008866,0.012289,-0.009848,0.001452,0.023535,0.004836,0.010960,0.041280,...,0.010892,0.007995,-0.035501,0.044304,0.026799,0.003959,0.001326,0.008671,0.028768,0.004361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08,-0.049368,0.018227,0.013549,0.005280,-0.010580,0.000493,0.009167,0.004975,0.004676,-0.013972,...,0.032909,0.037381,-0.045621,-0.055008,-0.035323,-0.001280,0.022186,0.023411,-0.006249,0.038324
2020-09,-0.006052,0.006131,-0.014598,-0.011446,-0.012887,-0.009621,-0.004355,-0.003496,-0.008132,-0.004987,...,-0.025945,-0.029365,0.001516,0.021054,0.031783,0.020368,-0.030054,-0.046113,-0.005784,0.006464
2020-10,-0.011785,-0.003378,-0.004676,-0.007752,0.007652,0.044951,0.056244,0.049368,0.050133,0.041393,...,0.030765,0.032177,-0.004541,-0.007067,-0.009132,-0.000454,-0.013322,-0.019826,0.004541,0.002010


In [4]:
data = db.get_table(library='comp', table='idxcst_his') #idxcst_his
data
####################################### saving ########################################
# data.to_hdf(wrds_data, 'idx_const_hist', mode='a', data_columns=True, complib='zlib')
#######################################################################################"

,gvkey,iid,gvkeyx,from,thru
0,001013,01,000208,1995-01-01,2003-12-21
1,001045,01,000006,1900-01-01,2012-01-04
2,001045,04,000006,2015-10-15,None
3,001045,04,000208,2014-12-22,2020-04-20
4,001055,01,000208,1995-01-01,1995-12-17
...,...,...,...,...,...
595,184996,01,000208,2013-07-15,None
596,199356,01,000208,2013-12-23,2017-05-03
597,205692,92,000208,2011-12-19,2013-11-19
598,211009,90,000208,2006-12-20,2012-12-11


In [5]:
# WRDS API has some useful methods:
# db.close()
# db.connection(),
# db.describe_table()
# db.get_table()
# db.list_tables()
# db.raw_sql()
# db.get_row_count()
# db.list_libraries()

# For exmaple:
# db.list_libraries()
# db.describe_table(library='crspm', table='idxcst_his')
# db.list_tables(library='comp')
db.list_tables(library='crsp')
# db.list_tables(library='djones')

['acti',
 'asia',
 'asib',
 'asic',
 'asio',
 'asix',
 'bmdebt',
 'bmheader',
 'bmpaymts',
 'bmquotes',
 'bmyield',
 'bndprt06',
 'bndprt12',
 'bxcalind',
 'bxdlyind',
 'bxmthind',
 'bxquotes',
 'bxyield',
 'cap',
 'ccm_lookup',
 'ccm_qvards',
 'ccmxpf_linktable',
 'ccmxpf_lnkhist',
 'ccmxpf_lnkrng',
 'ccmxpf_lnkused',
 'comphead',
 'comphist',
 'compmaster',
 'contact_info',
 'crsp_cik_map',
 'crsp_daily_data',
 'crsp_header',
 'crsp_monthly_data',
 'crsp_names',
 'crsp_portno_map',
 'crsp_ziman_daily_index',
 'crsp_ziman_monthly_index',
 'cs20yr',
 'cs5yr',
 'cs90d',
 'cst_hist',
 'daily_nav',
 'daily_nav_ret',
 'daily_returns',
 'dividends',
 'dport1',
 'dport2',
 'dport3',
 'dport4',
 'dport5',
 'dport6',
 'dport7',
 'dport8',
 'dport9',
 'dsbc',
 'dsbo',
 'dse',
 'dse62',
 'dse62delist',
 'dse62dist',
 'dse62exchdates',
 'dse62names',
 'dse62nasdin',
 'dse62shares',
 'dseall',
 'dseall62',
 'dsedelist',
 'dsedist',
 'dseexchdates',
 'dsenames',
 'dsenasdin',
 'dseshares',
 'dsf',


In [6]:
# Daily Data cannot be downloaded since the data size exceeds my storage

# tableofinterest = db.get_table(library='crsp', table='dsf')
# tableofinterest.head(20)

## Downloading the monthly stock data (msf) containing all 36,408 US american Stocks

In [ ]:
data_monthly = db.get_table(library='crsp', table='msf') #idxcst_his 
data_monthly.head(20)

In [ ]:
len(data_monthly.permno.unique())

In [ ]:
import datetime as dt
# len(data.permco.unique())

data_monthly['date'] = data_monthly['date'].astype(str)
# len(data_monthly.loc[(data_monthly.date>=('1963-01-01'))&(data_monthly.date<=('2020-12-31'))])
data_monthly = (data_monthly.loc[(data_monthly.date>=('1963-01-01'))&(data_monthly.date<=('2020-12-31'))])
data_monthly = data_monthly.groupby('permno').apply(lambda x: x.drop_duplicates('date')).reset_index(drop=True)
data_monthly 

In [11]:
####################################### saving ##################################################
# data_monthly.to_hdf(wrds_data, 'monthly_stock_file', mode='w', data_columns=True, complib='zlib')
# data_monthly.to_csv(wrds_data_raw_csv) 
############################################################################################################

## Pre alteration of dataset (sort values, format date, remove stocks with negative prices)

In [12]:
monthly_stock_data = pd.read_hdf(wrds_data, 'monthly_stock_file')

In [13]:
monthly_stock_data = monthly_stock_data.loc[:,['permno','date', 'prc', 'ret', 'shrout']]
monthly_stock_data.date = pd.to_datetime(monthly_stock_data.date)
monthly_stock_data.date = monthly_stock_data.date.dt.strftime('%Y-%m')
monthly_stock_data.sort_values(['permno','date'])
monthly_stock_data = monthly_stock_data.loc[(monthly_stock_data.date >=('1977-07'))&((monthly_stock_data.date <=('2020-12')))]
monthly_stock_data


,permno,date,prc,ret,shrout
0,10000.0,1985-12,NaN,NaN,NaN
1,13542.0,2016-10,23.319901,-0.029147,150.0
3,43481.0,2014-10,86.250000,0.163340,12754.0
4,58852.0,1981-04,-12.500000,0.063830,981.0
5,75549.0,2004-01,0.820000,0.051282,27622.0
...,...,...,...,...,...
4301634,70092.0,1988-01,9.500000,0.151515,27003.0
4301635,78156.0,2002-12,3.600000,-0.010989,18276.0
4301636,83455.0,1998-04,7.000000,-0.017544,16222.0
4301637,89056.0,2006-05,52.939999,-0.093863,20113.0


In [14]:
kept_permnos_1 =[]
permno_unique= monthly_stock_data.permno.unique()

for p in tqdm(permno_unique):
    if all(monthly_stock_data.loc[monthly_stock_data.permno==p].prc>0) ==True:
        kept_permnos_1.append(p)

monthly_stock_data_prc_abvzero =monthly_stock_data[monthly_stock_data.permno.isin(kept_permnos_1)] 
monthly_stock_data_prc_abvzero

100%|████████████████████████████████████████████████████████████████| 31801/31801 [03:38<00:00, 145.36it/s]


,permno,date,prc,ret,shrout
7048,27684.0,1977-07,18.750000,-0.032258,14274.0
7057,27684.0,1977-08,16.875000,-0.090667,14274.0
7066,27684.0,1977-09,17.500000,0.037037,14274.0
7075,27684.0,1977-10,16.125000,-0.078571,14274.0
7084,27684.0,1977-11,16.000000,0.003101,14274.0
...,...,...,...,...,...
4301089,92550.0,2014-12,41.869999,0.008916,150312.0
4301095,92550.0,2015-01,41.959999,0.002150,150312.0
4301101,92550.0,2015-02,45.450001,0.095806,150312.0
4301107,92550.0,2015-03,45.540001,0.001980,150318.0


In [15]:
len(monthly_stock_data_prc_abvzero.permno.unique())

624

## The problem with prices downloaded from WRDS

Over 18.000 stocks have prices that have a negative pre-signs, although when checking the returns it should be positive since all returns correspond to the price of t+1. Therefore all stock prices are changed to absolute values to maintain the large dataset (proof can found below)

In [16]:
monthly_stock_data['prc'] = abs(monthly_stock_data['prc'])

# Creation of datasets
* Dataset using stocks where returns and shares outstanding were constantly available during the time period from 1977 - 2020 (200)
* Dataset using stocks where returns and shares outstanding were constantly available during the time period from 1977 - 2020 where the values for stock prices are absolute values (480)
* Dataset using all stocks that traded between 1977 and 2020 where the values for stock prices are absolute values (33.000)

## Dataset with stocks that were actually traded each month from 1977/07 - 2020/12

* negative price stocks have been converted to absolute stock prices 

In [17]:
kept_permnos_2 =[]
permno_unique= monthly_stock_data.permno.unique()

for p in tqdm(permno_unique): 
    if (((('1977-07') in (monthly_stock_data.loc[monthly_stock_data.permno==p].date.sort_values().tolist()))==True) & (((monthly_stock_data.loc[monthly_stock_data.permno==p].date.sort_values().tolist()[-1]) == ('2020-12'))==True)):
        if len((monthly_stock_data.loc[monthly_stock_data.permno==p].date>=('1977-07'))&(monthly_stock_data.loc[monthly_stock_data.permno==p].date<=('2020-12'))) >=len(monthly_factors):
            kept_permnos_2.append(p)


100%|█████████████████████████████████████████████████████████████████| 31801/31801 [08:24<00:00, 63.03it/s]


In [18]:
len(kept_permnos_2)

520

In [19]:
monthly_stock_data_adj =  monthly_stock_data[monthly_stock_data.permno.isin(kept_permnos_2)]

In [20]:
monthly_stock_data_adj

,permno,date,prc,ret,shrout
3,43481.0,2014-10,86.250000,0.163340,12754.0
12,43481.0,2014-11,72.970001,-0.152580,12754.0
21,43481.0,2014-12,77.260002,0.058791,12754.0
30,43481.0,2015-01,70.089996,-0.092804,12754.0
39,43481.0,2015-02,72.500000,0.036097,12772.0
...,...,...,...,...,...
4301587,53373.0,2011-07,1.520000,-0.131429,22116.0
4301593,53373.0,2011-08,1.458900,-0.040197,22114.0
4301599,53373.0,2011-09,1.662800,0.139763,22107.0
4301605,53373.0,2011-10,1.340000,-0.194130,22107.0


In [21]:
nan_ret_values = monthly_stock_data_adj[monthly_stock_data_adj['ret'].isna()]
permnos_to_drop = list(nan_ret_values.permno.unique())

for p in permnos_to_drop: 
        idx =monthly_stock_data_adj.loc[monthly_stock_data_adj.permno==p].index
        monthly_stock_data_adj = monthly_stock_data_adj.drop(idx)

len(monthly_stock_data_adj.permno.unique())

458

In [22]:
print(len(monthly_stock_data_adj.permno.unique()))

458


## Creation of market cap and value weights

In [23]:
monthly_stock_data_adj.sort_values(by=['permno','date'],inplace=True)
monthly_stock_data_adj.groupby(['permno','date']).last().reset_index()


monthly_stock_data_adj['mktcap'] = monthly_stock_data_adj['prc']*monthly_stock_data_adj['shrout']
monthly_stock_data_adj.loc[:,'mktcapL'] = monthly_stock_data_adj.groupby('permno')['mktcap'].shift(-1)*int(1)
monthly_stock_data_adj.loc[:,'vweights'] = monthly_stock_data_adj.groupby(['date'])['mktcapL'].apply(lambda x: x/np.nansum(x)).replace(0.0, np.nan)
monthly_stock_data_adj.drop(columns=['mktcapL'], inplace=True)

monthly_stock_data_adj.sort_values(by=['permno','date'], inplace=True)
monthly_stock_data_adj

,permno,date,prc,ret,shrout,mktcap,vweights
85261,10065.0,1977-07,12.125000,0.000000,14548.0,1.763945e+05,0.000605
85270,10065.0,1977-08,12.000000,-0.010309,14548.0,1.745760e+05,0.000608
85279,10065.0,1977-09,12.000000,0.008333,14548.0,1.745760e+05,0.000633
85288,10065.0,1977-10,12.000000,0.000000,14548.0,1.745760e+05,0.000641
85297,10065.0,1977-11,12.625000,0.052083,14548.0,1.836685e+05,0.000644
...,...,...,...,...,...,...,...
2861121,84129.0,2020-08,32.160000,0.000924,163981.0,5.273629e+06,0.000462
2861129,84129.0,2020-09,29.219999,-0.091418,164009.0,4.792343e+06,0.000520
2861137,84129.0,2020-10,32.270000,0.104381,164012.0,5.292667e+06,0.000559
2861145,84129.0,2020-11,38.590000,0.206384,164012.0,6.329223e+06,0.000612


## Calculation of market return, excess returns and forward excess returns

In [24]:
def clc_market_return_day(df_month):
    df_month =df_month.dropna(subset=['shrout', 'prc', 'ret'])
    nom = df_month['shrout']*df_month['prc']*df_month['ret'] 
    den = df_month['shrout']*df_month['prc']
    return (nom.sum()/den.sum())

ret_mk = pd.DataFrame(monthly_stock_data_adj.groupby(['date']).apply(clc_market_return_day), columns = ['ret_mk'])
monthly_stock_data_adj = pd.merge(monthly_stock_data_adj, ret_mk,how='left', on=['date'])
monthly_stock_data_adj

,permno,date,prc,ret,shrout,mktcap,vweights,ret_mk
0,10065.0,1977-07,12.125000,0.000000,14548.0,1.763945e+05,0.000605,-0.011493
1,10065.0,1977-08,12.000000,-0.010309,14548.0,1.745760e+05,0.000608,-0.002968
2,10065.0,1977-09,12.000000,0.008333,14548.0,1.745760e+05,0.000633,-0.002997
3,10065.0,1977-10,12.000000,0.000000,14548.0,1.745760e+05,0.000641,-0.035402
4,10065.0,1977-11,12.625000,0.052083,14548.0,1.836685e+05,0.000644,0.044042
...,...,...,...,...,...,...,...,...
239071,84129.0,2020-08,32.160000,0.000924,163981.0,5.273629e+06,0.000462,0.047784
239072,84129.0,2020-09,29.219999,-0.091418,164009.0,4.792343e+06,0.000520,-0.017750
239073,84129.0,2020-10,32.270000,0.104381,164012.0,5.292667e+06,0.000559,-0.012986
239074,84129.0,2020-11,38.590000,0.206384,164012.0,6.329223e+06,0.000612,0.126480


In [25]:
monthly_stock_data_adj.loc[monthly_stock_data_adj.date==('1977-07')]

,permno,date,prc,ret,shrout,mktcap,vweights,ret_mk
0,10065.0,1977-07,12.125,0.000000,14548.0,176394.500,0.000605,-0.011493
522,10145.0,1977-07,47.625,-0.047500,28089.0,1337738.625,0.004113,-0.011493
1044,10516.0,1977-07,19.250,0.092199,29788.0,573419.000,0.002060,-0.011493
1566,10866.0,1977-07,20.625,0.024845,7303.0,150624.375,0.000541,-0.011493
2088,10874.0,1977-07,13.125,-0.051786,19828.0,260242.500,0.000936,-0.011493
...,...,...,...,...,...,...,...,...
236466,80128.0,1977-07,12.625,-0.151261,1770.0,22346.250,0.000077,-0.011493
236988,81294.0,1977-07,25.250,0.087234,1217.0,30729.250,0.000110,-0.011493
237510,82107.0,1977-07,26.750,0.048544,495.0,13241.250,0.000046,-0.011493
238032,83601.0,1977-07,24.875,-0.019704,3202.0,79649.750,0.000273,-0.011493


In [26]:
monthly_stock_data_adj.sort_values(by=['permno', 'date'],inplace=True)
monthly_stock_data_adj['monthly_exret'] = monthly_stock_data_adj['ret'] - monthly_stock_data_adj['ret_mk']
monthly_stock_data_adj['fwd_monthly_exret'] = monthly_stock_data_adj.groupby('permno')['monthly_exret'].shift(-1)
monthly_stock_data_adj = monthly_stock_data_adj.set_index('date') 

In [27]:
############################################# saving #########################################################
monthly_stock_data_adj.to_hdf(wrds_data, key='monthly_stock_data_full_range_abs_set', mode='a', data_columns=True, complib='zlib')
monthly_stock_data_adj.to_csv(wrds_data_abs_csv) 
############################################################################################################

## Dataset with stocks that were actually traded each month from 1977/07 - 2020/12

* negative price stocks have been removed which results in a loss of 33.000 stocks 

In [28]:
monthly_stock_data_prc_abvzero = monthly_stock_data_prc_abvzero[monthly_stock_data_prc_abvzero.permno.isin(kept_permnos_2)]

In [29]:
nan_ret_values = monthly_stock_data_prc_abvzero[monthly_stock_data_prc_abvzero['ret'].isna()]
permnos_to_drop = list(nan_ret_values.permno.unique())

for p in permnos_to_drop: 
        idx =monthly_stock_data_prc_abvzero.loc[monthly_stock_data_prc_abvzero.permno==p].index
        monthly_stock_data_prc_abvzero = monthly_stock_data_prc_abvzero.drop(idx)

len(monthly_stock_data_prc_abvzero.permno.unique())

200

## Creation of market cap and value weights

In [30]:
monthly_stock_data_prc_abvzero.sort_values(by=['permno','date'],inplace=True)
monthly_stock_data_prc_abvzero.groupby(['permno','date']).last().reset_index()


monthly_stock_data_prc_abvzero['mktcap'] = monthly_stock_data_prc_abvzero['prc']*monthly_stock_data_prc_abvzero['shrout']
monthly_stock_data_prc_abvzero.loc[:,'mktcapL'] = monthly_stock_data_prc_abvzero.groupby('permno')['mktcap'].shift(-1)*int(1)
monthly_stock_data_prc_abvzero.loc[:,'vweights'] = monthly_stock_data_prc_abvzero.groupby(['date'])['mktcapL'].apply(lambda x: x/np.nansum(x)).replace(0.0, np.nan)
monthly_stock_data_prc_abvzero.drop(columns=['mktcapL'], inplace=True)

monthly_stock_data_prc_abvzero.sort_values(by=['permno','date'], inplace=True)
monthly_stock_data_prc_abvzero

C:\Users\mauri\AppData\Local\Temp\ipykernel_14552\327171392.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_stock_data_prc_abvzero.sort_values(by=['permno','date'],inplace=True)
C:\Users\mauri\AppData\Local\Temp\ipykernel_14552\327171392.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_stock_data_prc_abvzero['mktcap'] = monthly_stock_data_prc_abvzero['prc']*monthly_stock_data_prc_abvzero['shrout']
C:\Users\mauri\AppData\Local\Temp\ipykernel_14552\327171392.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

,permno,date,prc,ret,shrout,mktcap,vweights
85261,10065.0,1977-07,12.125000,0.000000,14548.0,1.763945e+05,0.000658
85270,10065.0,1977-08,12.000000,-0.010309,14548.0,1.745760e+05,0.000662
85279,10065.0,1977-09,12.000000,0.008333,14548.0,1.745760e+05,0.000689
85288,10065.0,1977-10,12.000000,0.000000,14548.0,1.745760e+05,0.000699
85297,10065.0,1977-11,12.625000,0.052083,14548.0,1.836685e+05,0.000703
...,...,...,...,...,...,...,...
196703,59619.0,2020-08,55.220001,0.057652,42357.0,2.338954e+06,0.000319
196711,59619.0,2020-09,57.740002,0.045636,42357.0,2.445693e+06,0.000369
196720,59619.0,2020-10,65.430000,0.139938,42357.0,2.771419e+06,0.000364
196729,59619.0,2020-11,71.720001,0.096133,42357.0,3.037844e+06,0.000364


## Calculation of market return, excess returns and forward excess returns

In [31]:
ret_mk = pd.DataFrame(monthly_stock_data_prc_abvzero.groupby(['date']).apply(clc_market_return_day), columns = ['ret_mk'])
monthly_stock_data_prc_abvzero = pd.merge(monthly_stock_data_prc_abvzero, ret_mk,how='left', on=['date'])
monthly_stock_data_prc_abvzero

,permno,date,prc,ret,shrout,mktcap,vweights,ret_mk
0,10065.0,1977-07,12.125000,0.000000,14548.0,1.763945e+05,0.000658,-0.012553
1,10065.0,1977-08,12.000000,-0.010309,14548.0,1.745760e+05,0.000662,-0.002742
2,10065.0,1977-09,12.000000,0.008333,14548.0,1.745760e+05,0.000689,-0.004178
3,10065.0,1977-10,12.000000,0.000000,14548.0,1.745760e+05,0.000699,-0.035365
4,10065.0,1977-11,12.625000,0.052083,14548.0,1.836685e+05,0.000703,0.041593
...,...,...,...,...,...,...,...,...
104395,59619.0,2020-08,55.220001,0.057652,42357.0,2.338954e+06,0.000319,0.047415
104396,59619.0,2020-09,57.740002,0.045636,42357.0,2.445693e+06,0.000369,-0.017532
104397,59619.0,2020-10,65.430000,0.139938,42357.0,2.771419e+06,0.000364,-0.015114
104398,59619.0,2020-11,71.720001,0.096133,42357.0,3.037844e+06,0.000364,0.124409


In [32]:
monthly_stock_data_prc_abvzero.sort_values(by=['permno', 'date'],inplace=True)
monthly_stock_data_prc_abvzero['monthly_exret'] = monthly_stock_data_prc_abvzero['ret'] - monthly_stock_data_prc_abvzero['ret_mk']
monthly_stock_data_prc_abvzero['fwd_monthly_exret'] = monthly_stock_data_prc_abvzero.groupby('permno')['monthly_exret'].shift(-1)
monthly_stock_data_prc_abvzero = monthly_stock_data_prc_abvzero.set_index('date') 

In [33]:
############################################# saving #########################################################
monthly_stock_data_prc_abvzero.to_hdf(wrds_data, key='monthly_stock_data_full_range_true_set', mode='a', data_columns=True, complib='zlib')
monthly_stock_data_prc_abvzero.to_csv(wrds_data_true_csv) 
############################################################################################################

## Creation of complete data set including approx. 33.000 stocks 

* date formatting
* select respective time period starting from 1977 - 2020
* over 18.000 stocks have prices that have a negative pre-signs, although when checking the returns it should be positive since all returns correspond to the price of t+1. Therefore all stock prices are changed to absolute values to maintain the large dataset (proof can found below)

In [58]:
monthly_stock_data = pd.read_hdf(wrds_data, 'monthly_stock_file')

In [59]:
monthly_stock_data = monthly_stock_data.loc[:,['permno', 'date', 'prc', 'ret', 'shrout']]
monthly_stock_data.date = pd.to_datetime(monthly_stock_data.date)
monthly_stock_data.date = monthly_stock_data.date.dt.strftime('%Y-%m')
monthly_stock_data.sort_values(['permno','date'])
monthly_stock_data = monthly_stock_data.loc[(monthly_stock_data.date >=('1977-07'))&((monthly_stock_data.date <=('2020-12')))]

In [60]:
display(monthly_stock_data.loc[monthly_stock_data.prc<0])
len(monthly_stock_data.loc[monthly_stock_data.prc<0].permno.unique())

,permno,date,prc,ret,shrout
4,58852.0,1981-04,-12.5000,0.063830,981.0
9,10000.0,1986-01,-4.3750,NaN,3680.0
13,58852.0,1981-05,-12.1250,-0.030000,981.0
16,85403.0,1998-03,-17.6875,0.141129,32532.0
18,10000.0,1986-02,-3.2500,-0.257143,3680.0
...,...,...,...,...,...
4301548,83410.0,1999-03,-7.5000,0.025641,4143.0
4301563,53373.0,2009-05,-1.7850,0.231035,22198.0
4301569,53373.0,2009-06,-1.7100,-0.042017,22194.0
4301575,53373.0,2011-05,-1.7950,-0.065104,22118.0


18399

In [61]:
monthly_stock_data['prc'] = abs(monthly_stock_data.prc)

In [62]:
# monthly_stock_data_ffilled.dropna()
monthly_stock_data.sort_values(['permno','date'])
monthly_stock_data = monthly_stock_data.dropna()
monthly_stock_data

,permno,date,prc,ret,shrout
1,13542.0,2016-10,23.319901,-0.029147,150.0
3,43481.0,2014-10,86.250000,0.163340,12754.0
4,58852.0,1981-04,12.500000,0.063830,981.0
5,75549.0,2004-01,0.820000,0.051282,27622.0
6,79685.0,1998-04,27.000000,0.053659,4648.0
...,...,...,...,...,...
4301632,93435.0,2012-04,1.760000,-0.345725,23864.0
4301634,70092.0,1988-01,9.500000,0.151515,27003.0
4301635,78156.0,2002-12,3.600000,-0.010989,18276.0
4301636,83455.0,1998-04,7.000000,-0.017544,16222.0


In [63]:
monthly_stock_data.loc[monthly_stock_data.prc<0]

,permno,date,prc,ret,shrout


In [64]:
monthly_stock_data = monthly_stock_data.loc[(monthly_stock_data.date>=('1977-01'))&(monthly_stock_data.date<=('2020-12'))]

monthly_stock_data.dropna(inplace=True) 
monthly_stock_data.head(5)

,permno,date,prc,ret,shrout
1,13542.0,2016-10,23.319901,-0.029147,150.0
3,43481.0,2014-10,86.250000,0.163340,12754.0
4,58852.0,1981-04,12.500000,0.063830,981.0
5,75549.0,2004-01,0.820000,0.051282,27622.0
6,79685.0,1998-04,27.000000,0.053659,4648.0


## require that all stocks need to have at least 12 years of data 

In [65]:
permnos =monthly_stock_data.permno.unique() 

to_keep =[]
for p in tqdm(permnos):
    if len(monthly_stock_data.loc[monthly_stock_data.permno==p])>150:
        to_keep.append(p)
        
monthly_stock_data = monthly_stock_data.loc[monthly_stock_data.permno.isin(to_keep)]

100%|███████████████████████████████████████████████████████████████████████████| 31524/31524 [04:41<00:00, 112.03it/s]


In [66]:
len(monthly_stock_data.permno.unique())

8146

In [67]:
actuality_filter  = monthly_stock_data.loc[monthly_stock_data.date==('2020-11')]

monthly_stock_data_adj = monthly_stock_data.loc[monthly_stock_data.permno.isin(actuality_filter.permno.unique())]

In [68]:
len(monthly_stock_data_adj.permno.unique())

3359

## Creation of market cap and value weights

In [69]:
monthly_stock_data.sort_values(by=['permno','date'],inplace=True)
monthly_stock_data.groupby(['permno','date']).last().reset_index()

monthly_stock_data = monthly_stock_data.groupby('permno').apply(lambda x: x.drop_duplicates('date')).reset_index(drop=True)
monthly_stock_data.sort_values(by=['permno','date'], inplace=True)


monthly_stock_data['mktcap'] = monthly_stock_data['prc']*monthly_stock_data['shrout']
monthly_stock_data.loc[:,'mktcapL'] = monthly_stock_data.groupby('permno')['mktcap'].shift(-1)*int(1)
monthly_stock_data.loc[:,'vweights'] = monthly_stock_data.groupby(['date'])['mktcapL'].apply(lambda x: x/np.nansum(x)).replace(0.0, np.nan)
monthly_stock_data.drop(columns=['mktcapL'], inplace=True)

# monthly_stock_data = monthly_stock_data.groupby('permno').apply(lambda x: x.drop_duplicates('date')).reset_index(drop=True)
monthly_stock_data.sort_values(by=['permno','date'], inplace=True)
monthly_stock_data

,permno,date,prc,ret,shrout,mktcap,vweights
0,10001.0,1986-02,6.25000,0.020408,985.0,6156.2500,0.000003
1,10001.0,1986-03,6.31250,0.025200,985.0,6217.8125,0.000003
2,10001.0,1986-04,6.37500,0.009901,985.0,6279.3750,0.000003
3,10001.0,1986-05,6.31250,-0.009804,985.0,6217.8125,0.000003
4,10001.0,1986-06,6.12500,-0.013069,985.0,6033.1250,0.000003
...,...,...,...,...,...,...,...
2183965,93316.0,1999-10,6.87500,-0.035088,3082.0,21188.7500,0.000002
2183966,93316.0,1999-11,8.50000,0.236364,3082.0,26197.0000,0.000003
2183967,93316.0,1999-12,12.87500,0.514706,3082.0,39680.7500,0.000003
2183968,93316.0,2000-01,15.12500,0.174757,3082.0,46615.2500,0.000003


## Calculation of market return, excess returns and forward excess returns

In [70]:
def clc_market_return_day(df_month):
    df_month =df_month.dropna(subset=['shrout', 'prc', 'ret'])
    nom = df_month['shrout']*df_month['prc']*df_month['ret'] 
    den = df_month['shrout']*df_month['prc']
    return (nom.sum()/den.sum())

ret_mk = pd.DataFrame(monthly_stock_data.groupby(['date']).apply(clc_market_return_day), columns = ['ret_mk'])
monthly_stock_data = pd.merge(monthly_stock_data, ret_mk,how='left', on=['date'])
monthly_stock_data

,permno,date,prc,ret,shrout,mktcap,vweights,ret_mk
0,10001.0,1986-02,6.25000,0.020408,985.0,6156.2500,0.000003,0.080556
1,10001.0,1986-03,6.31250,0.025200,985.0,6217.8125,0.000003,0.059830
2,10001.0,1986-04,6.37500,0.009901,985.0,6279.3750,0.000003,-0.002530
3,10001.0,1986-05,6.31250,-0.009804,985.0,6217.8125,0.000003,0.053263
4,10001.0,1986-06,6.12500,-0.013069,985.0,6033.1250,0.000003,0.020472
...,...,...,...,...,...,...,...,...
2183965,93316.0,1999-10,6.87500,-0.035088,3082.0,21188.7500,0.000002,0.078024
2183966,93316.0,1999-11,8.50000,0.236364,3082.0,26197.0000,0.000003,0.045828
2183967,93316.0,1999-12,12.87500,0.514706,3082.0,39680.7500,0.000003,0.115025
2183968,93316.0,2000-01,15.12500,0.174757,3082.0,46615.2500,0.000003,-0.017687


In [71]:
monthly_stock_data.sort_values(by=['permno', 'date'],inplace=True)
monthly_stock_data['monthly_exret'] = monthly_stock_data['ret'] - monthly_stock_data['ret_mk']
monthly_stock_data['fwd_monthly_exret'] = monthly_stock_data.groupby('permno')['monthly_exret'].shift(-1)
monthly_stock_data = monthly_stock_data.set_index('date')

In [72]:
to_drop = monthly_stock_data.loc[monthly_stock_data.index==('2020-12')].index
monthly_stock_data = monthly_stock_data[~monthly_stock_data.index.isin(to_drop)]
monthly_stock_data

,permno,prc,ret,shrout,mktcap,vweights,ret_mk,monthly_exret,fwd_monthly_exret
date,,,,,,,,,
1986-02,10001.0,6.25000,0.020408,985.0,6156.2500,0.000003,0.080556,-0.060148,-0.034630
1986-03,10001.0,6.31250,0.025200,985.0,6217.8125,0.000003,0.059830,-0.034630,0.012431
1986-04,10001.0,6.37500,0.009901,985.0,6279.3750,0.000003,-0.002530,0.012431,-0.063067
1986-05,10001.0,6.31250,-0.009804,985.0,6217.8125,0.000003,0.053263,-0.063067,-0.033541
1986-06,10001.0,6.12500,-0.013069,985.0,6033.1250,0.000003,0.020472,-0.033541,0.039383
...,...,...,...,...,...,...,...,...,...
1999-10,93316.0,6.87500,-0.035088,3082.0,21188.7500,0.000002,0.078024,-0.113112,0.190535
1999-11,93316.0,8.50000,0.236364,3082.0,26197.0000,0.000003,0.045828,0.190535,0.399681
1999-12,93316.0,12.87500,0.514706,3082.0,39680.7500,0.000003,0.115025,0.399681,0.192445


In [73]:
monthly_stock_data.loc[monthly_stock_data.index==('2020-12')]

,permno,prc,ret,shrout,mktcap,vweights,ret_mk,monthly_exret,fwd_monthly_exret
date,,,,,,,,,


In [74]:
####################################### saving ############################################
monthly_stock_data.to_hdf(wrds_data, 'monthly_stock_data_complete_data', mode='a', data_columns=True, complib='zlib')
monthly_stock_data.to_csv(wrds_data_complete_abs_csv) 
############################################################################################################

In [75]:
monthly_stock_data = pd.read_csv(wrds_data_complete_abs_csv)

In [76]:
monthly_stock_data

,date,permno,prc,ret,shrout,mktcap,vweights,ret_mk,monthly_exret,fwd_monthly_exret
0,1986-02,10001.0,6.25000,0.020408,985.0,6156.2500,0.000003,0.080556,-0.060148,-0.034630
1,1986-03,10001.0,6.31250,0.025200,985.0,6217.8125,0.000003,0.059830,-0.034630,0.012431
2,1986-04,10001.0,6.37500,0.009901,985.0,6279.3750,0.000003,-0.002530,0.012431,-0.063067
3,1986-05,10001.0,6.31250,-0.009804,985.0,6217.8125,0.000003,0.053263,-0.063067,-0.033541
4,1986-06,10001.0,6.12500,-0.013069,985.0,6033.1250,0.000003,0.020472,-0.033541,0.039383
...,...,...,...,...,...,...,...,...,...,...
2180620,1999-10,93316.0,6.87500,-0.035088,3082.0,21188.7500,0.000002,0.078024,-0.113112,0.190535
2180621,1999-11,93316.0,8.50000,0.236364,3082.0,26197.0000,0.000003,0.045828,0.190535,0.399681
2180622,1999-12,93316.0,12.87500,0.514706,3082.0,39680.7500,0.000003,0.115025,0.399681,0.192445
2180623,2000-01,93316.0,15.12500,0.174757,3082.0,46615.2500,0.000003,-0.017687,0.192445,-0.070119


## Download of stocknames 

In [ ]:
import datetime as dt

stocknames = db.get_table(library ='crsp', table='stocknames')
stocknames = pd.DataFrame(stocknames)
stocknames['st_date'] = stocknames['st_date'].astype(str)
stocknames['end_date'] = stocknames['end_date'].astype(str)
stocknames.nameenddt = stocknames.nameenddt.astype(str)

stocknames = stocknames.loc[((stocknames.end_date>=('1963-01-01'))&(stocknames.st_date<=('2020-12-31')))]
stocknames = stocknames.loc[((stocknames.nameenddt>=('1963-01-01')))]
# len(stocknames.permno.unique())
# len(stocknames.permco.unique())
stocknames

In [ ]:
####################################### saving ############################################
stocknames.to_hdf(wrds_data, 'stocknames', mode='a', data_columns=True, complib='zlib')
###########################################################################################

## Daily data download 

In [3]:
import datetime as dt
comp = pd.read_csv(r'C:\Users\mauri\FSoF\Frankfurt School of Finance - Master Thesis\SP500_Index_Constitutes2020.csv')
comp['start']  = pd.to_datetime(comp['start'])
comp['ending'] = pd.to_datetime(comp['ending'])
comp = comp.loc[comp['ending']> dt.datetime(2010,12,31),:]
# comp.head()
len(comp.permno.unique())

734

In [7]:
#check
len(comp.permno.unique())
################################################  uncomment for data download #################################
sql_wrds = """
select distinct date, permno, cusip, ret, abs(prc) as prc, shrout, abs(prc)*shrout/1000 as mktcap    
from crsp.dsf                                                   
where permno in %(permno)s and date>'1998-12-31'
"""

prm = {'permno':tuple(comp.permno.unique())}

data_daily_sp = db.raw_sql(sql_wrds, params = prm)
data_daily_sp = data_daily_sp.dropna()
data_daily_sp = pd.merge(data_daily_sp, comp, how='left', on=['permno'])
data_daily_sp.date = pd.to_datetime(data_daily_sp.date)             
                   
                                      
# data_daily_sp.loc[:,'sp500'] = (data_daily_sp['date'] >= data_daily_sp['start']) & ((data_daily_sp['date'] <= data_daily_sp['ending']) |(data_daily_sp['date'] > data_daily_sp['ending'].max()))
data_daily_sp.loc[:,'sp500'] = 1*((data_daily_sp.date>data_daily_sp.start) & ((data_daily_sp.date<=data_daily_sp.ending) | (data_daily_sp.ending==dt.datetime(2021,1,1))))

data_daily_sp.sort_values(by=['permno', 'date'],inplace=True)
data_daily_sp.groupby(['permno','date']).last().reset_index()

data_daily_sp = data_daily_sp.loc[(data_daily_sp.date >=dt.datetime(1990,1,1))&(data_daily_sp.date<=dt.datetime(2021,1,1))]


In [10]:
data_daily_sp

,date,permno,cusip,ret,prc,shrout,mktcap,start,ending,sp500
521523,1999-01-04,10104.0,68389X10,-0.002899,43.000000,959712.0,41267.616000,1989-08-03,2020-12-31,1
230003,1999-01-05,10104.0,68389X10,0.030523,44.312500,959712.0,42527.238000,1989-08-03,2020-12-31,1
253780,1999-01-06,10104.0,68389X10,0.046544,46.375000,959712.0,44506.644000,1989-08-03,2020-12-31,1
207978,1999-01-07,10104.0,68389X10,-0.016173,45.625000,959712.0,43786.860000,1989-08-03,2020-12-31,1
268176,1999-01-08,10104.0,68389X10,0.013699,46.250000,959712.0,44386.680000,1989-08-03,2020-12-31,1
...,...,...,...,...,...,...,...,...,...,...
2733943,2020-12-24,93436.0,88160R10,0.024444,661.770020,947901.0,627292.463284,2020-12-21,2020-12-31,1
1286375,2020-12-28,93436.0,88160R10,0.002901,663.690002,947901.0,629112.417004,2020-12-21,2020-12-31,1
1832524,2020-12-29,93436.0,88160R10,0.003465,665.989990,947901.0,631292.577733,2020-12-21,2020-12-31,1
2751190,2020-12-30,93436.0,88160R10,0.043229,694.780029,947901.0,658582.684551,2020-12-21,2020-12-31,1


In [11]:
large_cap_filter = data_daily_sp.loc[data_daily_sp.date==('2020-12-30')]
data_daily_sp = data_daily_sp.loc[data_daily_sp.permno.isin(large_cap_filter.permno.unique())]
data_daily_sp

,date,permno,cusip,ret,prc,shrout,mktcap,start,ending,sp500
521523,1999-01-04,10104.0,68389X10,-0.002899,43.000000,959712.0,41267.616000,1989-08-03,2020-12-31,1
230003,1999-01-05,10104.0,68389X10,0.030523,44.312500,959712.0,42527.238000,1989-08-03,2020-12-31,1
253780,1999-01-06,10104.0,68389X10,0.046544,46.375000,959712.0,44506.644000,1989-08-03,2020-12-31,1
207978,1999-01-07,10104.0,68389X10,-0.016173,45.625000,959712.0,43786.860000,1989-08-03,2020-12-31,1
268176,1999-01-08,10104.0,68389X10,0.013699,46.250000,959712.0,44386.680000,1989-08-03,2020-12-31,1
...,...,...,...,...,...,...,...,...,...,...
2733943,2020-12-24,93436.0,88160R10,0.024444,661.770020,947901.0,627292.463284,2020-12-21,2020-12-31,1
1286375,2020-12-28,93436.0,88160R10,0.002901,663.690002,947901.0,629112.417004,2020-12-21,2020-12-31,1
1832524,2020-12-29,93436.0,88160R10,0.003465,665.989990,947901.0,631292.577733,2020-12-21,2020-12-31,1
2751190,2020-12-30,93436.0,88160R10,0.043229,694.780029,947901.0,658582.684551,2020-12-21,2020-12-31,1


In [6]:
# data_daily_sp['prc'] = abs(data_daily_sp.prc)

In [13]:
data_daily_sp = data_daily_sp.groupby('permno').apply(lambda x: x.drop_duplicates('date')).reset_index(drop=True)
data_daily_sp.sort_values(by=['permno','date'], inplace=True)


data_daily_sp['mktcap'] = data_daily_sp['prc']*data_daily_sp['shrout']
data_daily_sp.loc[:,'mktcapL'] = data_daily_sp.groupby('permno')['mktcap'].shift(-1)*int(1)
data_daily_sp.loc[:,'vweights'] = data_daily_sp.groupby(['date'])['mktcapL'].apply(lambda x: x/np.nansum(x)).replace(0.0, np.nan)
data_daily_sp.drop(columns=['mktcapL', 'start', 'ending', 'sp500','cusip'], inplace=True)

data_daily_sp.sort_values(by=['permno','date'], inplace=True)
data_daily_sp

,date,permno,ret,prc,shrout,mktcap,vweights
0,1999-01-04,10104.0,-0.002899,43.000000,959712.0,4.126762e+07,0.006532
1,1999-01-05,10104.0,0.030523,44.312500,959712.0,4.252724e+07,0.006686
2,1999-01-06,10104.0,0.046544,46.375000,959712.0,4.450664e+07,0.006569
3,1999-01-07,10104.0,-0.016173,45.625000,959712.0,4.378686e+07,0.006638
4,1999-01-08,10104.0,0.013699,46.250000,959712.0,4.438668e+07,0.006846
...,...,...,...,...,...,...,...
2803156,2020-12-24,93436.0,0.024444,661.770020,947901.0,6.272925e+08,0.018917
2803157,2020-12-28,93436.0,0.002901,663.690002,947901.0,6.291124e+08,0.019026
2803158,2020-12-29,93436.0,0.003465,665.989990,947901.0,6.312926e+08,0.019819
2803159,2020-12-30,93436.0,0.043229,694.780029,947901.0,6.585827e+08,0.020254


In [14]:
def clc_market_return_day(df):
    df =df.dropna(subset=['shrout', 'prc', 'ret'])
    nom = df['shrout']*df['prc']*df['ret'] 
    den = df['shrout']*df['prc']
    return (nom.sum()/den.sum())

In [15]:
ret_mk = pd.DataFrame(data_daily_sp.groupby(['date']).apply(clc_market_return_day), columns = ['ret_mk'])
data_daily_sp = pd.merge(data_daily_sp, ret_mk,how='left', on=['date'])
data_daily_sp

,date,permno,ret,prc,shrout,mktcap,vweights,ret_mk
0,1999-01-04,10104.0,-0.002899,43.000000,959712.0,4.126762e+07,0.006532,-0.001681
1,1999-01-05,10104.0,0.030523,44.312500,959712.0,4.252724e+07,0.006686,0.013046
2,1999-01-06,10104.0,0.046544,46.375000,959712.0,4.450664e+07,0.006569,0.023368
3,1999-01-07,10104.0,-0.016173,45.625000,959712.0,4.378686e+07,0.006638,0.002016
4,1999-01-08,10104.0,0.013699,46.250000,959712.0,4.438668e+07,0.006846,0.003493
...,...,...,...,...,...,...,...,...
2803156,2020-12-24,93436.0,0.024444,661.770020,947901.0,6.272925e+08,0.018917,0.003599
2803157,2020-12-28,93436.0,0.002901,663.690002,947901.0,6.291124e+08,0.019026,0.009018
2803158,2020-12-29,93436.0,0.003465,665.989990,947901.0,6.312926e+08,0.019819,-0.002224
2803159,2020-12-30,93436.0,0.043229,694.780029,947901.0,6.585827e+08,0.020254,0.001785


In [16]:
data_daily_sp.sort_values(by=['permno', 'date'],inplace=True)
data_daily_sp['daily_exret'] = data_daily_sp['ret'] - data_daily_sp['ret_mk']
data_daily_sp['fwd_daily_exret'] = data_daily_sp.groupby('permno')['daily_exret'].shift(-1)
data_daily_sp = data_daily_sp.set_index('date')


In [19]:
data_daily_sp.dropna(inplace=True)

In [21]:
data_daily_sp.loc[:,'fwd1w_ret_mk'] = data_daily_sp.groupby(['permno'])['ret_mk'].\
                            apply(lambda z: z.rolling(window = 5).\
                                apply(lambda x: np.exp(np.sum(np.log(1+x)))-1 )).shift(-5)

data_daily_sp.loc[:,'fwd1w_ret'] = data_daily_sp.groupby(['permno'])['ret'].\
                            apply(lambda z: z.rolling(window = 5).\
                                apply(lambda x: np.exp(np.sum(np.log(1+x)))-1 )).shift(-5)

data_daily_sp['fwd_weekly_exret'] = data_daily_sp['fwd1w_ret'] - data_daily_sp['fwd1w_ret_mk'] 

data_daily_sp.loc[:,'fwd1m_ret_mk'] = data_daily_sp.groupby(['permno'])['ret_mk'].\
                            apply(lambda z: z.rolling(window = 21).\
                                apply(lambda x: np.exp(np.sum(np.log(1+x)))-1 )).shift(-21)

data_daily_sp.loc[:,'fwd1m_ret'] = data_daily_sp.groupby(['permno'])['ret'].\
                            apply(lambda z: z.rolling(window = 21).\
                                apply(lambda x: np.exp(np.sum(np.log(1+x)))-1 )).shift(-21)

data_daily_sp['fwd_monthly_exret'] = data_daily_sp['fwd1m_ret'] - data_daily_sp['fwd1m_ret_mk'] 


In [22]:
to_drop = data_daily_sp.loc[data_daily_sp.index==('2020-12-31')].index
data_daily_sp = data_daily_sp[~data_daily_sp.index.isin(to_drop)]
data_daily_sp

,permno,ret,prc,shrout,mktcap,vweights,ret_mk,daily_exret,fwd_daily_exret,fwd1w_ret_mk,fwd1w_ret,fwd_weekly_exret,fwd1m_ret_mk,fwd1m_ret,fwd_monthly_exret
date,,,,,,,,,,,,,,,
1999-01-04,10104.0,-0.002899,43.000000,959712.0,4.126762e+07,0.006532,-0.001681,-0.001218,0.017477,0.032244,0.097384,0.065140,0.050669,0.412791,0.362122
1999-01-05,10104.0,0.030523,44.312500,959712.0,4.252724e+07,0.006686,0.013046,0.017477,0.023176,-0.000911,0.036671,0.037582,0.018203,0.315938,0.297735
1999-01-06,10104.0,0.046544,46.375000,959712.0,4.450664e+07,0.006569,0.023368,0.023176,-0.018188,-0.027541,-0.013477,0.014064,-0.012597,0.211590,0.224187
1999-01-07,10104.0,-0.016173,45.625000,959712.0,4.378686e+07,0.006638,0.002016,-0.018188,0.010206,-0.045428,-0.021918,0.023511,-0.010463,0.257534,0.267998
1999-01-08,10104.0,0.013699,46.250000,959712.0,4.438668e+07,0.006846,0.003493,0.010206,0.030049,-0.026411,0.018919,0.045330,-0.035241,0.181081,0.216322
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-23,93436.0,0.008808,645.979980,947901.0,6.123251e+08,0.019028,0.001085,0.007723,0.020845,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-24,93436.0,0.024444,661.770020,947901.0,6.272925e+08,0.018917,0.003599,0.020845,-0.006116,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-28,93436.0,0.002901,663.690002,947901.0,6.291124e+08,0.019026,0.009018,-0.006116,0.005690,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
ff_factors = pd.read_csv(r'C:\Users\mauri\FSoF\Frankfurt School of Finance - Master Thesis\Master Thesis - Data\Factors\Fama-French\FF Factor Daily\F-F_Research_Data_5_Factors_2x3_daily.CSV', header=2)
ff_factors =  ff_factors.rename(columns={'Unnamed: 0':'date'})
ff_factors['date'] = ff_factors['date'].astype(str) 
ff_factors = ff_factors.loc[(ff_factors.date>=('1999-01-01'))&(ff_factors.date<=('2021-01-01'))]
ff_factors = ff_factors[['date','RF']]
ff_factors['date'] = pd.to_datetime(ff_factors['date'])
ff_factors.set_index('date',inplace=True)
ff_factors = ff_factors/100
ff_factors.reset_index(inplace=True)
ff_factors

,date,RF
0,1999-01-04,0.00019
1,1999-01-05,0.00019
2,1999-01-06,0.00019
3,1999-01-07,0.00019
4,1999-01-08,0.00019
...,...,...
5531,2020-12-24,0.00000
5532,2020-12-28,0.00000
5533,2020-12-29,0.00000
5534,2020-12-30,0.00000


In [57]:
test = data_daily_sp.copy()

data_daily_sp = pd.merge(data_daily_sp, ff_factors, how='left', on='date')
data_daily_sp['daily_exret_rf'] = data_daily_sp['ret']-data_daily_sp['RF']
data_daily_sp['fwd_daily_exret_rf'] = data_daily_sp.groupby('permno')['daily_exret_rf'].apply(lambda x: x.shift(-1))
data_daily_sp

,date,permno,ret,prc,shrout,mktcap,vweights,ret_mk,daily_exret,fwd_daily_exret,fwd1w_ret_mk,fwd1w_ret,fwd_weekly_exret,fwd1m_ret_mk,fwd1m_ret,fwd_monthly_exret,RF,daily_exret_rf,fwd_daily_exret_rf
0,1999-01-04,10104.0,-0.002899,43.000000,959712.0,4.126762e+07,0.006532,-0.001681,-0.001218,0.017477,0.032244,0.097384,0.065140,0.050669,0.412791,0.362122,0.00019,-0.003089,0.030333
1,1999-01-05,10104.0,0.030523,44.312500,959712.0,4.252724e+07,0.006686,0.013046,0.017477,0.023176,-0.000911,0.036671,0.037582,0.018203,0.315938,0.297735,0.00019,0.030333,0.046354
2,1999-01-06,10104.0,0.046544,46.375000,959712.0,4.450664e+07,0.006569,0.023368,0.023176,-0.018188,-0.027541,-0.013477,0.014064,-0.012597,0.211590,0.224187,0.00019,0.046354,-0.016363
3,1999-01-07,10104.0,-0.016173,45.625000,959712.0,4.378686e+07,0.006638,0.002016,-0.018188,0.010206,-0.045428,-0.021918,0.023511,-0.010463,0.257534,0.267998,0.00019,-0.016363,0.013509
4,1999-01-08,10104.0,0.013699,46.250000,959712.0,4.438668e+07,0.006846,0.003493,0.010206,0.030049,-0.026411,0.018919,0.045330,-0.035241,0.181081,0.216322,0.00019,0.013509,0.020080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2802572,2020-12-23,93436.0,0.008808,645.979980,947901.0,6.123251e+08,0.019028,0.001085,0.007723,0.020845,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.008808,0.024444
2802573,2020-12-24,93436.0,0.024444,661.770020,947901.0,6.272925e+08,0.018917,0.003599,0.020845,-0.006116,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.024444,0.002901
2802574,2020-12-28,93436.0,0.002901,663.690002,947901.0,6.291124e+08,0.019026,0.009018,-0.006116,0.005690,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.002901,0.003465
2802575,2020-12-29,93436.0,0.003465,665.989990,947901.0,6.312926e+08,0.019819,-0.002224,0.005690,0.041444,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.003465,0.043229


In [ ]:
data_daily_sp_backup = data_daily_sp.copy(deep=True)


In [59]:
y = data_daily_sp.loc[(data_daily_sp.date>=('2020-01-01'))&(data_daily_sp.date<=('2020-12-31'))]
y = y.groupby('permno').apply(lambda x: len(x))
y

permno
10104.0    252
10107.0    252
10138.0    252
10145.0    252
10516.0    252
          ... 
93096.0    252
93132.0    252
93422.0    252
93429.0    252
93436.0    252
Length: 584, dtype: int64

In [60]:
len(data_daily_sp.permno.unique())

584

In [61]:
############################################## saving     ######################################################
data_daily_sp.to_csv(daily_wrds_data_csv)
################################################################################################################

In [62]:
data_daily_sp = pd.read_csv(daily_wrds_data_csv)

In [63]:
data_daily_sp

,Unnamed: 0,date,permno,ret,prc,shrout,mktcap,vweights,ret_mk,daily_exret,fwd_daily_exret,fwd1w_ret_mk,fwd1w_ret,fwd_weekly_exret,fwd1m_ret_mk,fwd1m_ret,fwd_monthly_exret,RF,daily_exret_rf,fwd_daily_exret_rf
0,0,1999-01-04,10104.0,-0.002899,43.000000,959712.0,4.126762e+07,0.006532,-0.001681,-0.001218,0.017477,0.032244,0.097384,0.065140,0.050669,0.412791,0.362122,0.00019,-0.003089,0.030333
1,1,1999-01-05,10104.0,0.030523,44.312500,959712.0,4.252724e+07,0.006686,0.013046,0.017477,0.023176,-0.000911,0.036671,0.037582,0.018203,0.315938,0.297735,0.00019,0.030333,0.046354
2,2,1999-01-06,10104.0,0.046544,46.375000,959712.0,4.450664e+07,0.006569,0.023368,0.023176,-0.018188,-0.027541,-0.013477,0.014064,-0.012597,0.211590,0.224187,0.00019,0.046354,-0.016363
3,3,1999-01-07,10104.0,-0.016173,45.625000,959712.0,4.378686e+07,0.006638,0.002016,-0.018188,0.010206,-0.045428,-0.021918,0.023511,-0.010463,0.257534,0.267998,0.00019,-0.016363,0.013509
4,4,1999-01-08,10104.0,0.013699,46.250000,959712.0,4.438668e+07,0.006846,0.003493,0.010206,0.030049,-0.026411,0.018919,0.045330,-0.035241,0.181081,0.216322,0.00019,0.013509,0.020080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2802572,2802572,2020-12-23,93436.0,0.008808,645.979980,947901.0,6.123251e+08,0.019028,0.001085,0.007723,0.020845,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.008808,0.024444
2802573,2802573,2020-12-24,93436.0,0.024444,661.770020,947901.0,6.272925e+08,0.018917,0.003599,0.020845,-0.006116,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.024444,0.002901
2802574,2802574,2020-12-28,93436.0,0.002901,663.690002,947901.0,6.291124e+08,0.019026,0.009018,-0.006116,0.005690,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.002901,0.003465
2802575,2802575,2020-12-29,93436.0,0.003465,665.989990,947901.0,6.312926e+08,0.019819,-0.002224,0.005690,0.041444,NaN,NaN,NaN,NaN,NaN,NaN,0.00000,0.003465,0.043229
